# 任务 3：探索和查询来自 PySpark 内核的数据

在此笔记本中，您将执行以下操作：

- 了解如何使用 Studio 笔记本以可视化方式浏览 Amazon EMR 集群、对其进行身份验证并连接到该集群。
- 使用 Spark ML 探索和查询数据。
- 使用 Spark UI 监控 Spark 作业。

### 任务 3.1：会话信息

因为您正在使用 PySpark 内核，所以可以使用 PySpark 魔法 %%info 来显示当前会话信息。

In [ ]:
%%info

### 任务 3.2：探索和查询数据

使用 PySpark 内核时，连接到 EMR 集群后会自动创建 SparkContext 和 HiveContext。您可以使用 HiveContext 来查询 Hive 表中的数据，并使其在 Spark 数据帧中可用。

使用 HiveContext 来查询 Hive 并观察数据库和表。

In [ ]:
#query-hive

sqlContext = HiveContext(sqlContext)

dbs = sqlContext.sql("show databases")
dbs.show()

tables = sqlContext.sql("show tables")
tables.show()

查询成人数据表并将数据传入 Spark 数据帧。

In [ ]:
#load-data

adult_df = sqlContext.sql("select * from adult_data").cache()

使用数据帧观察形状并查看数据集的前五行。

In [ ]:
#view-shape

print((adult_df.count(), len(adult_df.columns)))

#Show first 5 rows 
adult_df.head(5)

为了获得更清晰的输出，请将 Spark 数据帧转换为 Pandas 数据帧。

In [ ]:
#convert-dataframe

adult_df.limit(5).toPandas()

一些机器学习 (ML) 算法（例如线性回归）需要数值特征。您在本实验中使用的成人数据集包括分类特征，例如 **workclass**、**education**、**occupation**、**marital status**、**relationship**、**race** 和 **sex**。

下面的代码块演示了如何使用 StringIndexer 和 OneHotEncoderEstimator 将分类变量转换为一组值为 0 和 1 的数值变量。

- StringIndexer 可将字符串值列转换为标签索引列。
- OneHotEncoderEstimator 可将分类索引列映射到二进制向量列，每行最多一个“1”，表示该行的分类索引。

Spark 中的独热编码是一个分为两步的流程。首先使用 **StringIndexer**，然后使用 **OneHotEncoder**。

有关更多信息，请参阅 [StringIndexer](https://spark.apache.org/docs/latest/ml-features.html#stringindexer) 和 [OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder)。

In [ ]:
#convert-variables

from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

categorical_variables = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

VectorAssembler 类将多个列作为输入。它输出一个包含值数组的列。

有关此汇编器的更多信息，请参阅 [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)。

In [ ]:
#vector-assembler

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

管道是转换器和估算器的有序列表。您可以定义管道来实现自动化并确保应用于数据集的转换的可重复性。在此步骤中，定义管道，然后将其应用于数据集。

类似于 **StringIndexer**，管道是**估算器**。pipeline.fit() 方法会返回 **PipelineModel**，这是**转换器**。

有关机器学习管道的更多信息，请参阅[管道](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline)。

In [ ]:
#pyspark-pipelines

from pyspark.ml import Pipeline

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=indexers + [encoder, assembler])

# Define the pipeline model.
pipelineModel = pipeline.fit(adult_df)

# Apply the pipeline model to the dataset.
adult_df = pipelineModel.transform(adult_df)

查看在上一步中创建的所有不同的列。

In [ ]:
#print-schema

adult_df.printSchema()

应用转换后，单个列包含一个数组，其中有每个已编码的分类变量。

In [ ]:
#view-categorical-features

adult_df.select('categorical-features').show(truncate=False)

现在，编码目标标签。

In [ ]:
#encode-target

indexer = StringIndexer(inputCol='income', outputCol='label')

adult_df = indexer.fit(adult_df).transform(adult_df)

### 任务 3.3：使用 Spark UI 进行监控和调试

在本部分中，您将使用 Spark UI 监控和检查在前面的步骤中运行的 Spark 任务的性能。

获取当前 Spark 会话信息。

In [ ]:
%%info

您可以看到 **Spark UI** 和**驱动程序日志**的超链接。在本实验中，**驱动程序日志**链接无效。**Spark UI** 预签名 URL 是在连接到 EMR 集群时生成的。选择此链接会将您转至 Spark UI，从而在 Web 浏览器中检查 Spark 任务运行情况。这些指标对优化性能很有帮助。

下面是 Spark 服务器中的一些重要功能：
- **Jobs**（任务）选项卡显示该 Spark 应用程序中所有 Spark 任务的状态。
- 在 <b>Summary</b>（摘要）部分下，**Event Timeline**（事件时间线）部分显示运行的各个阶段。
- **Completed Jobs**（已完成的任务）部分以表格形式显示。在 **Completed Jobs**（已完成的任务）部分下，您可以选择一项任务，从而查看该任务中子任务的阶段信息。
- 使用 **DAG Visualization**（DAG 可视化），您可以探索之前运行的子任务。与 **Event Timeline**（事件时间线）视图一样，使用 **DAG Visualization**（DAG 可视化），您可以选择一个阶段并展开该阶段中的详细信息。

### 总结

恭喜！ 您已成功连接到 EMR 集群，并使用 PySpark 探索和查询了数据。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。